In [1]:
# -*- coding: utf-8 -*- 
# %reset -f
"""
@author: Hiromasa Kaneko
"""
# Demonstration of Genetic Algorithm-based process Variables and Dynamics Selection
# using Support Vector Regression (GAVDSSVR)

import random

import numpy as np
import pandas as pd
from deap import base
from deap import creator
from deap import tools
from sklearn import model_selection
from sklearn import svm

# settings
max_dynamics_considered = 50
number_of_areas = 5
max_width_of_areas = 20

number_of_process_variables = 20

number_of_population = 100
number_of_generation = 150

dynamics_span = 1
svr_c_2_range = (-5, 10)
svr_epsilon_2_range = (-10, 0)
svr_gamma_2_range = (-20, 10)

fold_number = 5
probability_of_crossover = 0.5
probability_of_mutation = 0.2
threshold_of_variable_selection = 0.5

# load and pre-process dataset
dataset = pd.read_csv('debutanizer_y_measurement_span_10.csv')
# dataset = pd.read_csv( 'debutanizer.csv' )
dataset = np.array(dataset)
if max_dynamics_considered:
    dataset_with_dynamics = np.empty((dataset.shape[0] - max_dynamics_considered, 0))
    dataset_with_dynamics = np.append(dataset_with_dynamics, dataset[max_dynamics_considered:, 0:1], axis=1)
    for x_variable_number in range(dataset.shape[1] - 1):
        dataset_with_dynamics = np.append(dataset_with_dynamics,
                                          dataset[max_dynamics_considered:, x_variable_number + 1:x_variable_number + 2],
                                          axis=1)
        for time_delay_number in range(int(np.floor(max_dynamics_considered / dynamics_span))):
            dataset_with_dynamics = np.append(dataset_with_dynamics, dataset[max_dynamics_considered - (
                        time_delay_number + 1) * dynamics_span:-(time_delay_number + 1) * dynamics_span,
                                                                     x_variable_number + 1:x_variable_number + 2],
                                              axis=1)
else:
    dataset_with_dynamics = dataset

x_train_with_999 = dataset_with_dynamics[:, 1:]
y_train_with_999 = dataset_with_dynamics[:, 0]
x_train = x_train_with_999[y_train_with_999 != 999, :]
y_train = y_train_with_999[y_train_with_999 != 999]

# autoscaling
autoscaled_x_train = (x_train - x_train.mean(axis=0)) / x_train.std(axis=0, ddof=1)
autoscaled_y_train = (y_train - y_train.mean()) / y_train.std(ddof=1)

# GAVDSSVR
creator.create('FitnessMax', base.Fitness, weights=(1.0,))  # for minimization, set weights as (-1.0,)
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
min_boundary = np.zeros(number_of_areas * 2 + 3)
max_boundary = np.ones(number_of_areas * 2 + 3) * x_train.shape[1]
max_boundary[np.arange(1, number_of_areas * 2, 2)] = max_width_of_areas
min_boundary[-3] = svr_c_2_range[0]
min_boundary[-2] = svr_epsilon_2_range[0]
min_boundary[-1] = svr_gamma_2_range[0]
max_boundary[-3] = svr_c_2_range[1]
max_boundary[-2] = svr_epsilon_2_range[1]
max_boundary[-1] = svr_gamma_2_range[1]


def create_ind_uniform(min_boundary, max_boundary):
    index = []
    for min, max in zip(min_boundary, max_boundary):
        index.append(random.uniform(min, max))
    return index


toolbox.register('create_ind', create_ind_uniform, min_boundary, max_boundary)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.create_ind)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)


def evalOneMax(individual):
    individual_array = np.array(individual)
    individual_array_wavelength = np.array(np.floor(individual_array[0:number_of_areas * 2]), dtype=int)

    first_number_of_process_variables = np.arange(0, autoscaled_x_train.shape[1], max_dynamics_considered + 1)
    selected_x_variable_numbers = np.zeros(0, dtype=int)
    for area_number in range(number_of_areas):
        check_of_two_process_variables_selected = (first_number_of_process_variables - individual_array_wavelength[
            2 * area_number]) * (first_number_of_process_variables - individual_array_wavelength[2 * area_number] -
                                 individual_array_wavelength[2 * area_number + 1])
        flag = np.where(check_of_two_process_variables_selected < 0)[0]
        if len(flag) > 0:
            individual_array_wavelength[2 * area_number + 1] = first_number_of_process_variables[flag[0]] - \
                                                               individual_array_wavelength[2 * area_number]
        flag = np.where(first_number_of_process_variables - individual_array_wavelength[2 * area_number] -
                        individual_array_wavelength[2 * area_number + 1] == 0)[0]
        if len(flag) > 0:
            individual_array_wavelength[2 * area_number + 1] = first_number_of_process_variables[flag[0]] - \
                                                               individual_array_wavelength[2 * area_number]

        if individual_array_wavelength[2 * area_number] + individual_array_wavelength[2 * area_number + 1] <= \
                autoscaled_x_train.shape[1]:
            selected_x_variable_numbers = np.r_[
                selected_x_variable_numbers, np.arange(individual_array_wavelength[2 * area_number],
                                                       individual_array_wavelength[2 * area_number] +
                                                       individual_array_wavelength[2 * area_number + 1])]
        else:
            selected_x_variable_numbers = np.r_[
                selected_x_variable_numbers, np.arange(individual_array_wavelength[2 * area_number],
                                                       autoscaled_x_train.shape[1])]

    selected_autoscaled_x_train = autoscaled_x_train[:, selected_x_variable_numbers]
    if len(selected_x_variable_numbers):
        # cross-validation
        model_in_cv = svm.SVR(kernel='rbf', C=2 ** round(individual_array[-3]),
                              epsilon=2 ** round(individual_array[-2]), gamma=2 ** round(individual_array[-1]))
        estimated_y_train_in_cv = model_selection.cross_val_predict(model_in_cv, selected_autoscaled_x_train,
                                                                    autoscaled_y_train, cv=fold_number)
        estimated_y_train_in_cv = estimated_y_train_in_cv * y_train.std(ddof=1) + y_train.mean()
        value = 1 - sum((y_train - estimated_y_train_in_cv) ** 2) / sum((y_train - y_train.mean()) ** 2)
    else:
        value = -999

    return value,


toolbox.register('evaluate', evalOneMax)
toolbox.register('mate', tools.cxTwoPoint)
toolbox.register('mutate', tools.mutFlipBit, indpb=0.05)
toolbox.register('select', tools.selTournament, tournsize=3)

# random.seed(100)
random.seed()
pop = toolbox.population(n=number_of_population)

print('Start of evolution')

fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

print('  Evaluated %i individuals' % len(pop))

for generation in range(number_of_generation):
    print('-- Generation {0} --'.format(generation + 1))

    offspring = toolbox.select(pop, len(pop))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < probability_of_crossover:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < probability_of_mutation:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    print('  Evaluated %i individuals' % len(invalid_ind))

    pop[:] = offspring
    fits = [ind.fitness.values[0] for ind in pop]

    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x * x for x in fits)
    std = abs(sum2 / length - mean ** 2) ** 0.5

    print('  Min %s' % min(fits))
    print('  Max %s' % max(fits))
    print('  Avg %s' % mean)
    print('  Std %s' % std)

print('-- End of (successful) evolution --')

best_individual = tools.selBest(pop, 1)[0]
best_individual_array = np.array(best_individual)
best_individual_array_wavelength = np.array(np.floor(best_individual_array[0:number_of_areas * 2]), dtype=int)

first_number_of_process_variables = np.arange(0, autoscaled_x_train.shape[1], max_dynamics_considered)
selected_x_variable_numbers = np.zeros(0, dtype=int)
for area_number in range(number_of_areas):
    check_of_two_process_variables_selected = (first_number_of_process_variables - best_individual_array_wavelength[
        2 * area_number]) * (first_number_of_process_variables - best_individual_array_wavelength[2 * area_number] -
                             best_individual_array_wavelength[2 * area_number + 1])
    flag = np.where(check_of_two_process_variables_selected < 0)[0]
    if len(flag) > 0:
        best_individual_array_wavelength[2 * area_number + 1] = first_number_of_process_variables[flag[0]] - \
                                                                best_individual_array_wavelength[2 * area_number]
    flag = np.where(first_number_of_process_variables - best_individual_array_wavelength[2 * area_number] -
                    best_individual_array_wavelength[2 * area_number + 1] == 0)[0]
    if len(flag) > 0:
        best_individual_array_wavelength[2 * area_number + 1] = first_number_of_process_variables[flag[0]] - \
                                                                best_individual_array_wavelength[2 * area_number]

    if best_individual_array_wavelength[2 * area_number] + best_individual_array_wavelength[2 * area_number + 1] <= \
            autoscaled_x_train.shape[1]:
        selected_x_variable_numbers = np.r_[
            selected_x_variable_numbers, np.arange(best_individual_array_wavelength[2 * area_number],
                                                   best_individual_array_wavelength[2 * area_number] +
                                                   best_individual_array_wavelength[2 * area_number + 1])]
    else:
        selected_x_variable_numbers = np.r_[
            selected_x_variable_numbers, np.arange(best_individual_array_wavelength[2 * area_number],
                                                   autoscaled_x_train.shape[1])]

print('Selected variables : %s, %s' % (selected_x_variable_numbers, best_individual.fitness.values))
print('C : 2 ** {0}'.format(round(best_individual_array[-3])))
print('Epsilon : 2 ** {0}'.format(round(best_individual_array[-2])))
print('Gamma : 2 ** {0}'.format(round(best_individual_array[-1])))

C:\Users\Akitaka\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\Akitaka\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\Akitaka\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Start of evolution
  Evaluated 100 individuals
-- Generation 1 --
  Evaluated 60 individuals
  Min -0.6935958804424496
  Max 0.6378121293089103
  Avg 0.031253216884029295
  Std 0.18253720758842681
-- Generation 2 --
  Evaluated 53 individuals
  Min -0.15930725961360603
  Max 0.6389046938700058
  Avg 0.12978483332317356
  Std 0.20172227908040094
-- Generation 3 --
  Evaluated 56 individuals
  Min -0.8915184891360695
  Max 0.6780429549412998
  Avg 0.23304585272194633
  Std 0.2756390134261998
-- Generation 4 --
  Evaluated 62 individuals
  Min -0.13509220824894985
  Max 0.6869912653894334
  Avg 0.4154263307668514
  Std 0.21449583368316366
-- Generation 5 --
  Evaluated 49 individuals
  Min -0.03646147769001096
  Max 0.7063780407994522
  Avg 0.5542137412635431
  Std 0.154087844136874
-- Generation 6 --
  Evaluated 62 individuals
  Min -0.01988492978654577
  Max 0.7225609641557964
  Avg 0.5909879817718842
  Std 0.16259067415133704
-- Generation 7 --
  Evaluated 58 individuals
  Min -0.02612

  Evaluated 54 individuals
  Min 0.5689506842039562
  Max 0.7494821843427222
  Avg 0.744840515717318
  Std 0.025709983171278924
-- Generation 57 --
  Evaluated 60 individuals
  Min -0.01713055495576077
  Max 0.7494821843427222
  Avg 0.7270967526127958
  Std 0.11089486907006571
-- Generation 58 --
  Evaluated 53 individuals
  Min -0.01713055495576077
  Max 0.7494821843427222
  Avg 0.7348435374047149
  Std 0.08531141780503258
-- Generation 59 --
  Evaluated 59 individuals
  Min -0.030514899860933475
  Max 0.7494821843427222
  Avg 0.72310564231854
  Std 0.1328883648761677
-- Generation 60 --
  Evaluated 58 individuals
  Min -0.01713055495576077
  Max 0.7494821843427222
  Avg 0.7311000412531874
  Std 0.08359506669894569
-- Generation 61 --
  Evaluated 56 individuals
  Min 0.11708939869531199
  Max 0.7494821843427222
  Avg 0.7309287298033967
  Std 0.08946809375677331
-- Generation 62 --
  Evaluated 57 individuals
  Min -0.013897036411237629
  Max 0.7494821843427222
  Avg 0.7386550449194443


  Evaluated 61 individuals
  Min -0.01713055495576077
  Max 0.7494821843427222
  Avg 0.7227758095692817
  Std 0.11216068096566109
-- Generation 112 --
  Evaluated 65 individuals
  Min -0.12123143547869031
  Max 0.7494821843427222
  Avg 0.7232784377260129
  Std 0.12108875391322513
-- Generation 113 --
  Evaluated 59 individuals
  Min -0.01713055495576077
  Max 0.7494821843427222
  Avg 0.7292296804992944
  Std 0.08688115181997892
-- Generation 114 --
  Evaluated 51 individuals
  Min 0.5392519641924312
  Max 0.7494821843427222
  Avg 0.7388070551622044
  Std 0.03594438270367214
-- Generation 115 --
  Evaluated 58 individuals
  Min 0.5465312770781368
  Max 0.7494821843427222
  Avg 0.7456066103983457
  Std 0.02332422653208833
-- Generation 116 --
  Evaluated 65 individuals
  Min 0.5689506842039562
  Max 0.7494821843427222
  Avg 0.7440814762919254
  Std 0.02337909067088645
-- Generation 117 --
  Evaluated 62 individuals
  Min -0.01713055495576077
  Max 0.7494821843427222
  Avg 0.7385957162299